In [1]:
{
  "aquatic_mammals": [4, 30, 55, 72, 95],
  "fish": [1, 32, 67, 73, 91],
  "flowers": [54, 62, 70, 82, 92],
  "food_containers": [9, 10, 16, 28, 61],
  "fruit_and_vegetables": [0, 51, 53, 57, 83],
  "household_electrical_devices": [22, 39, 40, 86, 87],
  "household_furniture": [5, 20, 25, 84, 94],
  "insects": [6, 7, 14, 18, 24],
  "large_carnivores": [3, 42, 43, 88, 97],
  "large_man-made_outdoor_things": [12, 17, 37, 68, 76],
  "large_natural_outdoor_scenes": [23, 33, 49, 60, 71],
  "large_omnivores_and_herbivores": [15, 19, 21, 31, 38],
  "medium-sized_mammals": [34, 63, 64, 66, 75],
  "non-insect_invertebrates": [26, 45, 77, 78, 99],
  "people": [2, 11, 35, 46, 98],
  "reptiles": [27, 29, 44, 78, 93],
  "small_mammals": [36, 50, 65, 74, 80],
  "trees": [47, 52, 56, 59, 96],
  "vehicles_1": [8, 13, 48, 58, 90],
  "vehicles_2": [41, 69, 81, 85, 89]
}

{'aquatic_mammals': [4, 30, 55, 72, 95],
 'fish': [1, 32, 67, 73, 91],
 'flowers': [54, 62, 70, 82, 92],
 'food_containers': [9, 10, 16, 28, 61],
 'fruit_and_vegetables': [0, 51, 53, 57, 83],
 'household_electrical_devices': [22, 39, 40, 86, 87],
 'household_furniture': [5, 20, 25, 84, 94],
 'insects': [6, 7, 14, 18, 24],
 'large_carnivores': [3, 42, 43, 88, 97],
 'large_man-made_outdoor_things': [12, 17, 37, 68, 76],
 'large_natural_outdoor_scenes': [23, 33, 49, 60, 71],
 'large_omnivores_and_herbivores': [15, 19, 21, 31, 38],
 'medium-sized_mammals': [34, 63, 64, 66, 75],
 'non-insect_invertebrates': [26, 45, 77, 78, 99],
 'people': [2, 11, 35, 46, 98],
 'reptiles': [27, 29, 44, 78, 93],
 'small_mammals': [36, 50, 65, 74, 80],
 'trees': [47, 52, 56, 59, 96],
 'vehicles_1': [8, 13, 48, 58, 90],
 'vehicles_2': [41, 69, 81, 85, 89]}

In [2]:
import pandas as pd
from tqdm import tqdm

path = "/gscratch/cse/lee0618/cifar-100/meta/pairs.csv"
df = pd.read_csv(path)

md_path = "/gscratch/cse/lee0618/cifar-100/meta/train.csv"
md = pd.read_csv(md_path)

print(len(df))
superclass_agreement = df.loc[df["superclass_1"] == df["superclass_2"]]
label_agreement = df.loc[df["label_1"] == df["label_2"]]
# for i in tqdm(range(len(df))):
#     assert md.loc[md["id"] == df.loc[i, "id_1"]].iloc[0]["group"] == 1
#     assert md.loc[md["id"] == df.loc[i, "id_2"]].iloc[0]["group"] == 2
print(len(superclass_agreement) / len(df))
print(len(label_agreement) / len(df))

138250
0.7347269439421338
0.6201591320072333


In [4]:
import os
import sys
import torch

sys.path.insert(0, "/gscratch/jamiemmt/andersonlee/image-distributionally-robust-data-join/src/models")
sys.path.insert(0, "/gscratch/jamiemmt/andersonlee/image-distributionally-robust-data-join/src")
from models.resnet_autoencoder import ResNet, get_configs

configs, bottleneck = get_configs()
model = ResNet(configs=configs, bottleneck=bottleneck, num_classes=100, pretrained=False)

x = torch.rand((2, 3, 224, 224))
model(x)

torch.Size([2, 2048, 7, 7])
torch.Size([2, 2048, 1, 1])


tensor([[ 1.2813e+00, -2.9571e+00,  6.6158e-01, -1.2060e+00, -1.5886e+00,
         -2.1522e-01, -9.2093e-01, -3.7148e-01,  2.2315e+00,  9.3347e-01,
         -3.0956e-01,  2.3489e+00, -1.3072e+00, -4.3904e+00,  2.2242e+00,
         -5.4975e-01, -3.0329e-01, -1.2797e+00,  1.4816e+00, -1.0300e+00,
          1.8652e+00,  1.4706e+00, -9.5255e-01,  4.4246e-01, -2.2785e-02,
          1.1444e+00, -6.2914e-01, -2.2332e+00, -5.3125e-01,  1.0125e+00,
          6.5147e-01,  7.6569e-01, -7.9201e-01,  1.4220e+00, -7.5579e-01,
          6.9312e-01, -4.3726e-01,  5.1090e-01,  1.0528e+00, -1.1137e+00,
          5.5941e-01, -1.8125e+00,  1.3415e-01,  2.7232e-01,  4.9868e-01,
         -1.5633e+00, -7.0223e-02, -5.3900e-03,  4.8588e-01,  1.3900e+00,
          4.3117e-01,  1.9867e+00, -1.4902e+00,  4.8150e-01, -7.1117e-01,
         -4.6741e-01, -3.2917e-01, -1.5299e+00,  1.6215e+00, -2.1221e+00,
         -4.5868e-01,  2.9233e+00,  9.1697e-01,  6.4795e-02, -8.8701e-01,
          9.3192e-01,  1.9998e-01, -1.